In [72]:
import requests
from IPython.display import JSON

DEBUG = False

url = f"https://docker78406-saas-kit.jcloud.ik-server.com"

if DEBUG:
    import os
    from dotenv import load_dotenv

    load_dotenv()
    PORT = os.getenv("WEBBFF_SERVICE_PORT")

    url = f"http://localhost:{PORT}"

# Default headers (extended with authorization when necessary)
headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

client = requests.session()

## Authentication service

In [73]:
# Login
payload = {
    'email': 'simon@smartnuance.com',
    'password': 'f00bartest',
    'instance': 'smartnuance.com'
}
response = client.post(f'{url}/auth/login', json=payload, headers=headers)
print(response.status_code)

login_res = response.json()
JSON(login_res)


200


<IPython.core.display.JSON object>

In [68]:
# Refresh tokens
payload = {
    'refreshToken': login_res["refreshToken"],
}
response = client.post(f'{url}/auth/refresh', json=payload, headers=headers)
print(response.status_code)

refresh_res = response.json()
login_res["accessToken"] = refresh_res["accessToken"]
JSON(refresh_res)


200


<IPython.core.display.JSON object>

In [69]:
# Revoke all tokens for a user
payload = {
    'email': 'simon@smartnuance.com',
}
auth_header = {
    'Authorization': f'Bearer {login_res["accessToken"]}',
}
response = client.delete(f'{url}/auth/revoke/all', json=payload, headers=dict(headers, **auth_header))
print(response.status_code)


200


## Interact with event service

In [70]:
# Create event
payload = {
    'title': 'Bachata',
    'locationName': 'Ponto',
}
auth_header = {
    'Authorization': f'Bearer {login_res["accessToken"]}',
    'role': 'event organizer',
    'instance': 'c5263570ono4ui8qfhgg',
}

response = client.put(f'{url}/event/workshop', json=payload, headers=dict(headers, **auth_header))
print(response.status_code)
workshop = response.json()
JSON(workshop)


201


<IPython.core.display.JSON object>

In [71]:
# List events
auth_header = {
    'Authorization': f'Bearer {login_res["accessToken"]}',
    'role': 'event organizer',
    'instance': 'c5263570ono4ui8qfhgg',
}
response = client.get(f'{url}/event/workshop/list', headers=dict(headers, **auth_header))
print(response.status_code)
JSON(response.json())


200


<IPython.core.display.JSON object>